In [76]:
from pyspark.sql import SparkSession

spark_session = SparkSession.builder\
.master("spark://192.168.2.70:7077") \
.appName("ludvigwesterholm_B")\
.config("spark.dynamicAllocation.enabled", True)\
.config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
.config("spark.shuffle.service.enabled", True)\
.config("spark.dynamicAllocation.executorIdleTimeout","30s")\
.config("spark.cores.max", 4)\
.getOrCreate()

In [77]:
#B.1 reading the csv
df = spark_session.read.csv("hdfs://192.168.2.70:9000/parking-citations.csv")
df.show()

+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+--------------------+-----------+---------+---------+------------------+-----------------+--------------------+
|          _c0|                 _c1|       _c2|     _c3|        _c4|           _c5|              _c6| _c7| _c8|       _c9| _c10|                _c11| _c12|  _c13|          _c14|                _c15|       _c16|     _c17|     _c18|              _c19|             _c20|                _c21|
+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+--------------------+-----------+---------+---------+------------------+-----------------+--------------------+
|Ticket number|          Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|     

In [78]:
#B.2 printing the schema
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)



In [79]:
#B.3 coutning number of rows
df.count()

13077725

In [80]:
#B.4 counting number of partitions
df.rdd.getNumPartitions()

16

In [81]:
#B.5 dropping columns
df = df.drop('_c7', '_c17', '_c18')

In [99]:
#B.6 finding biggest fine
from pyspark.sql.types import FloatType

df = df.withColumn("fines", df["_c16"].cast(FloatType()))

fines = df.select("fines")

fines = fines.dropna()

max_fine = fines.rdd.max()[0]

df.where(df["fines"]==max_fine).count()

626

In [116]:
#B.7 
makes = df.select("_c8")
makes = makes.dropna()

my_dict = makes.rdd.countByKey()

In [122]:
sorted_dict = dict(sorted(my_dict.items(), key=lambda x: x[1], reverse=True))
dict_keys = list(sorted_dict.keys())

for x in range(20):
    print(f"Make: {dict_keys[x]} \t Frequency: {sorted_dict[dict_keys[x]]}")

Make: TOYT 	 Frequency: 2150768
Make: HOND 	 Frequency: 1479996
Make: FORD 	 Frequency: 1116235
Make: NISS 	 Frequency: 945133
Make: CHEV 	 Frequency: 892676
Make: BMW 	 Frequency: 603092
Make: MERZ 	 Frequency: 543298
Make: VOLK 	 Frequency: 432030
Make: HYUN 	 Frequency: 404917
Make: DODG 	 Frequency: 391686
Make: LEXS 	 Frequency: 368420
Make: KIA 	 Frequency: 328155
Make: JEEP 	 Frequency: 316300
Make: AUDI 	 Frequency: 255395
Make: MAZD 	 Frequency: 242344
Make: OTHR 	 Frequency: 205546
Make: GMC 	 Frequency: 184889
Make: INFI 	 Frequency: 174315
Make: CHRY 	 Frequency: 159948
Make: SUBA 	 Frequency: 154640
